In [2]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# РЈРєР°Р¶РёС‚Рµ РІР°С€ email (РѕР±СЏР·Р°С‚РµР»СЊРЅРѕ РґР»СЏ РёСЃРїРѕР»СЊР·РѕРІР°РЅРёСЏ Entrez)
Entrez.email = "your.email@example.com"

In [3]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [4]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

35227


In [5]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [6]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [9]:
# add_histone = (
#     "INSERT INTO histone "
#     "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
#     "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
# )
# add_histone_description = (
#     "INSERT INTO histone_description "
#     "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
#     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
# )
add_publication = (
    "INSERT INTO publication "
    "(id, title, doi, author, year) "
    "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
)
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
add_histone_has_publication = (
    "INSERT INTO histone_has_publication "
    "(histone_id, publication_id) "
    "VALUES (%s, %s)"
)

In [ ]:
data_histone_description = {
    "summary": None,
    "taxonomy": None,
    "genes": None,
    "evolution": None, 
    "expression": None,
    "knock_out": None,
    "function": None,
    "sequence": None,
    "localization": None,
    "deposition": None,
    "structure": None,
    "interactions": None,
    "disease": None,
    "caveats": None,
}

In [10]:
def get_taxonomy_data(record):
    import re
    import sys

    taxonomy_data = {}
    taxonomy_data["organism"] = record.annotations["organism"]
    try:
        for a in record.features[0].qualifiers["db_xref"]:
            text = re.search("(\S+):(\S+)", a).group(1)
            taxid = re.search("(\S+):(\S+)", a).group(2)
            if text == "taxon":
                print(f"Fetched taxid from NCBI {taxid}")
                taxonomy_data["taxonomy_id"] = int(taxid)
            else:
                continue
    except:
        print("!!!!!!Unable to get TAXID for this record setting it to 1")
        taxonomy_data["taxonomy_id"] = 1  # unable to identify

    lineage = dict()
    for i in range(10):
        try:
            handle = Entrez.efetch(
                id=taxonomy_data["taxonomy_id"], db="taxonomy", retmode="xml"
            )
            tax_data = Entrez.read(handle)
            lineage = {
                d["Rank"]: d["ScientificName"]
                for d in tax_data[0]["LineageEx"]
                if d["Rank"] in ["class", "phylum"]
            }
            break
        except:
            print(
                "Unexpected error: {}, Retrying, attempt {}".format(
                    sys.exc_info()[0], i
                )
            )
            if i == 9:
                print(
                    f"FATAL ERROR could not get class and phylum from NCBI after 10 attempts for taxid:{taxonomy_data['taxonomy_id']}. Will add None for class and phylum!"
                )
            else:
                continue
    taxonomy_data["phylum"] = lineage.get("phylum", None)
    taxonomy_data["class"] = lineage.get("class", None)
    if taxonomy_data["phylum"] is not None:
        taxonomy_data["phylum"] = str(taxonomy_data["phylum"])
    if taxonomy_data["class"] is not None:
        taxonomy_data["class"] = str(taxonomy_data["class"])
    return taxonomy_data

In [8]:
query = "SELECT * FROM alternative_name " "WHERE name='cid'"
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,65,cid,None,None,None,cenH3_(Plants)


In [14]:
query = "SELECT * FROM publication "
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["id"].str.startswith("hara")]

,id,title,doi,author,year,pubmed_id


In [85]:
query = (
    "SELECT * FROM histone as h LEFT JOIN histone_description as hd "
    "ON h.description = hd.id "
    # "WHERE h.id = 'H3.6_(Homo_sapiens)' "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df.iloc[:,0].str.contains("H3.6")]

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
17,cenH3.6_(Repleta),variant,repleta group,32321,245.0,cenH3_(Drosophilidae),245.0,cenH3.6_(Repleta) is a recently evolved centr...,None,None,...,None,None,None,None,None,None,None,None,None,None


In [28]:
df["genes"].values[0]

'CENP-A nucleosomes are essential for chromosome segregation. A functional kinetochore interacts with active centromeric chromatin reached with CENP-A to form the mitotic spindle. Notably, in certain holocentric organisms, CENP-A appears dispensable for meiotic chromosome segregation while remaining essential for mitosis. As demonstrated in Caenorhabditis elegans, CENP-A loading is specifically eliminated following meiosis I, and RNAi-mediated depletion of CENP-A during meiosis fails to disrupt proper chromosome segregation [monen_differential_2005]. This stands in striking contrast to mitotic divisions, where CENP-A is absolutely required for kinetochore assembly and faithful chromosome segregation. Moreover, C. elegans and C. remanei possess a second cenH3 histone gene that likely performs specialized functions distinct from canonical CENP-A roles [monen_separase_2015]. For more detailed information see the "gene" section.'

In [70]:
query = "SELECT * FROM histone_description "
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
# df
for s in df.apply(lambda row: ''.join(row.astype(str)), axis=1):
    if "henikoff_heterochromatic_2000" in s:
        print(s)
        print("---------------")

In [41]:
query = "SHOW TABLES; "
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,Tables_in_db_name
0,alternative_name
1,histone
2,histone_description
3,histone_has_publication
4,publication
5,sequence
6,sequence_has_publication


In [11]:
query = (
    "SELECT s.accession, s.variant, s.organism, p.* FROM sequence s LEFT JOIN "
    "sequence_has_publication sp ON s.accession = sp.sequence_accession LEFT JOIN "
    "publication p ON sp.publication_id = p.id "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["variant"].str.startswith("H2A.Q", na=False)]

,accession,variant,organism,id,title,doi,author,year,pubmed_id
1407,ELK02218.1,H2A.Q_(Eutheria),Pteropus alecto,29549088,None,None,None,None,None
2367,HISTDB_H2A_Q_0,H2A.Q_(Eutheria),Cercopithecus aethiops,29549088,None,None,None,None,None
2368,HISTDB_H2A_Q_1,H2A.Q_(Eutheria),Pantholops hodgsonii,29549088,None,None,None,None,None
2369,HISTDB_H2A_Q_10,H2A.Q_(Eutheria),Ursus maritimus,29549088,None,None,None,None,None
2370,HISTDB_H2A_Q_11,H2A.Q_(Eutheria),Cercocebus atys,29549088,None,None,None,None,None
2371,HISTDB_H2A_Q_12,H2A.Q_(Eutheria),Odobenus rosmarus divergens,29549088,None,None,None,None,None
2372,HISTDB_H2A_Q_13,H2A.Q_(Eutheria),Ceratotherium simum,29549088,None,None,None,None,None
2373,HISTDB_H2A_Q_14,H2A.Q_(Eutheria),Bos mutus,29549088,None,None,None,None,None
2374,HISTDB_H2A_Q_15,H2A.Q_(Eutheria),Pan troglodytes,29549088,None,None,None,None,None
2375,HISTDB_H2A_Q_16,H2A.Q_(Eutheria),Bos taurus,29549088,None,None,None,None,None


# To Do H1

## <span style="color:black">Update description of H2B.V_(Trypanosoma)</span>

### <span style="color:black">Update expression</span>

```H2B.V transcript levels peak during the S phase of the cell cycle, similarly to canonical histones, whereas protein abundance progressively accumulates as the cycle progresses. A significant increase in H2B.V levels is observed during the initial stages of metacyclogenesis, with maximum abundance detected in metacyclic trypomastigotes [roson_new_2025].```

### <span style="color:black">Update function</span>

```The role of H2B.V appears to involve the regulation of parasite differentiation and virulence processes [roson_new_2025].```

### <span style="color:black">Update localization</span>

```H2B.V, together with H4.V, marks new transcription initiation and termination sites. H2B.V localizes to divergent strand-switch regions, which demarcate transcription initiation sites, as well as near certain tRNA loci [roson_new_2025].```

### <span style="color:black">Update knock_out</span>

```Parasites with heterozygous knockout of H2B.V exhibit increased differentiation into metacyclic forms and enhanced invasion of mammalian host cells [roson_new_2025].```

### <span style="color:black">Update interections</span>

```H2B.V dimerizes with the histone variant H2A.Z. The H2B.VвЂ“H2A.Z dimers are more unstable compared to canonical H2BвЂ“H2A dimers, thereby promoting a more accessible chromatin configuration permissive for transcription initiation [roson_new_2025].```

## <span style="color:black">Update description of H3.V_(Trypanosoma)</span>

### <span style="color:black">Update summary</span>

```H3.V is a histone variant characterized so far in some trypanosomatids. It shares ~45-60% sequence identity with canonical H3. H3.V is not essential for viability. Although H3.V associated with the mitotic spindle and enriched at telomeres it is not essential for mini-chromosome segregation, telomere maintenance or transcriptional silencing at the telomere-proximal expression sites [lowell_variant_2004, anderson_kinetoplastid-specific_2013]. While the general structure of H3.V resembles that of CenH3, it does not possess the specific sequence features required to be classified as a CenH3 variant [lowell_variant_2004].```

### <span style="color:black">Update function</span>

```H3.V plays a critical role in the regulation of antigenic variation. Together with H4.V they are involved in the positioning and expression of variant surface glycoprotein (VSG) genes, which the parasite Trypanosoma brucei switches to avoid immune detection [muller_genome_2018, schulz_base_2016]. H3.V is important for transcription termination and expression of downstream genes. The trypanosomatid-specific hyper-modified DNA base J and H3.V collaboratively influence the termination process resulting in read-through transcription and increased expression of downstream genes [siegel_four_2009, reynolds_histone_2016, schulz_base_2016, mcdonald_localization_2022].```

### <span style="color:black">Update localization</span>

```In Trypanosoma brucei, H3.V, together with H4.V, is enriched at transcription termination sites and in telomeric regions [anderson_kinetoplastid-specific_2013, roson_new_2025].```

### <span style="color:black">Update knock_out</span>

```While H3.V is dispensable for parasite viability, its deletion impairs variant surface glycoprotein (VSG) silencing. However, simultaneous deletion of H3.V, H4.V, and DNA base J leads to critical alterations in cell growth and replication [roson_new_2025].```

## <span style="color:black">Update description of cH1.5</span>

### <span style="color:black">Update summary</span>

```cH1.5 is a replication dependent linker histone located within large histone gene clusters in mammals. cH1.5 plays a critical role in chromatin organization, thereby influencing cellular development and homeostasis [behrends_linker_2020].```

### <span style="color:black">Update expression</span>

```cH1.5 is present across various cell types. As demonstrated on human cells, its expression level changes during differentiation: for instance, it decreases during the differentiation of NT2 cells into neural cells but increases during the dedifferentiation of keratinocytes into induced pluripotent stem cells [behrends_linker_2020].```

## <span style="color:black">Update description of cH1.5_(Homo_sapiens)</span>

### <span style="color:black">Update summary</span>

```cH1.5_(Homo_sapiens) is a replication dependent linker histone in human. СЃH1.5 plays a critical role in maintaining centromere integrity and is essential for sustaining appropriate transcriptional programs in differentiated cell types in human cells [saha_linker_2025, li_dynamic_2012].```

### <span style="color:black">Update expression</span>

```cH1.5 is present across various human cell types. Notably, it serves as the predominant H1 variant in astrocyte-lineage SVGp12 cells and in glioblastoma multiforme cells [saha_linker_2025]. cH1.5 expression is downregulated during cellular differentiation. Compared to normal differentiated somatic cells, H1.5 exhibits higher expression levels in both pluripotent cells and fibroblasts [li_dynamic_2012].```

### <span style="color:black">Update function</span>

```cH1.5 is essential for the regulation of mitotic integrity and for sustaining appropriate transcriptional programs in differentiated cell types [saha_linker_2025, li_dynamic_2012].```

### <span style="color:black">Update localization</span>

```In differentiated cells, cH1.5 forms enrichment blocks in both genic and intergenic regions, preferentially associating with genes encoding membrane and membrane-associated proteins [li_dynamic_2012]. Moreover, cH1.5 is enriched at centromeric regions and coвЂ‘localizes with the centromeric histone CENPвЂ‘A in human, as confirmed by immunofluorescence and chromatin immunoprecipitation assays [saha_linker_2025]. cH1.5 binding is also associated with gene repression, chromatin compaction and promoting the inclusion of alternatively spliced exons into the mature mRNA [li_dynamic_2012, behrends_linker_2020].```

>It is also enriched around splicing sites, particularly on genes with alternative splicing, where its binding promotes the inclusion of alternatively spliced exons into the mature mRNA (see behrends_linker_2020).

### <span style="color:black">Update knock_out</span>

```cH1.5 depletion leads to decreased cell growth, increased chromatin accessibility and deregulation of gene expression [li_dynamic_2012, behrends_linker_2020]. Additionally, knockdown of cH1.5 results in the loss of centromeric О±вЂ‘satellite DNA transcription, a reduction in de novo CENPвЂ‘A loading, and the accumulation of mitotic defects [saha_linker_2025].```

### <span style="color:black">Update structure</span>

```cH1.5 binds to CENPвЂ‘A nucleosomes in a nonвЂ‘canonical, dyadвЂ‘independent manner [saha_linker_2025].```

### <span style="color:black">Update interections</span>

```cH1.5 directly interacts with CENPвЂ‘AвЂ‘containing nucleosomes both in vitro and in vivo [saha_linker_2025].```

### <span style="color:black">Update disease</span>

```cH1.5 can serve as a valuable prognostic marker in cancer patients. Its expression level is decreased in many cancer cell lines, while being highly overexpressed in glioblastoma multiforme [saha_linker_2025, behrends_linker_2020, li_dynamic_2012]. Furthermore, cH1.5 mutations are associated with follicular lymphoma and altered chromatin states. Impairment of cH1.5 function may contribute to chromosomal instability and the development of malignancies, particularly in brain cells[saha_linker_2025].```

## <span style="color:black">Update description of cH1.1</span>

### <span style="color:black">Update summary</span>

```cH1.1 is a replication dependent linker histone located within large histone gene clusters in mammals. cH1.1 plays an important role in the early stages of embryogenesis [funaya_involvement_2023].```

### <span style="color:black">Update function</span>

```cH1.1 plays a critical role in the early stages of embryogenesis. cH1.1 is absent in human differentiated keratinocytes and fibroblasts but is re-expressed upon reprogramming, suggesting that it is specific to pluripotent cells [terme_histone_2011]. In mice, it is essential for preimplantation development, particularly at the morula and blastocyst stages [funaya_involvement_2023].```

### <span style="color:black">Update expression</span>

```cH1.1 expression predominates in early embryonic cells compared to other replication-dependent linker histone variants. In mice, cH1.1вЂ”alongside H1.8вЂ”exhibits the highest expression levels at the one- and two-cell stages of embryonic development [funaya_involvement_2023, funaya_linker_2018]. Although its expression declines by the four-cell stage, it remains elevated through the blastocyst stage. In human cells, СЃH1.1 has also been shown to be highly expressed in pluripotent cellsвЂ”including embryonic and induced pluripotent stem cellsвЂ”and absent in differentiated keratinocytes and fibroblasts [terme_histone_2011].```

## <span style="color:black">Update description of cH1.1_(Homo_sapiens)</span>

### <span style="color:black">Update summary</span>

```sdfa```



# Close connections

In [35]:
cursor.close()
conn.close()
tunnel.stop()